In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import all necessary modules first
import aicons.bayesbrainGPT.bayes_brain_ref
import aicons.bayesbrainGPT.utility_function.utility_base
import aicons.bayesbrainGPT.utility_function.linear_utility
import aicons.bayesbrainGPT.utility_function
import aicons.definitions.aicon

# Now we can reload them
importlib.reload(aicons.bayesbrainGPT.bayes_brain_ref)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)
importlib.reload(aicons.bayesbrainGPT.utility_function)  # Reload the package first
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT)
importlib.reload(aicons)

# Import the classes we need
from aicons.definitions.aicon import AIcon

In [2]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


### Defining Action Space

In [3]:
# Define an action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [4]:
print(aicon.brain.action_space.raw_print()) 

dimensions: ['google_budget: [0.0, 1000.0] step=100.0', 'facebook_budget: [0.0, 1000.0] step=100.0']
constraints: ['<function create_marketing_ads_space.<locals>.budget_sum_constraint at 0x3108b4180>']
actions: [{'google_budget': 100.0, 'facebook_budget': 900.0}, {'google_budget': 900.0, 'facebook_budget': 100.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 400.0, 'facebook_budget': 600.0}, {'google_budget': 100.0, 'facebook_budget': 900.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 900.0, 'facebook_budget': 100.0}, {'google_budget': 400.0, 'facebook_budget': 600.0}, {'google_budget': 300.0, 'facebook_budget': 700.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 900.0, 'facebook_budget': 100.0}, {'google_budget': 600.0, 'facebook_budget': 400.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 0.0, 'facebook_budget': 1000.0}, {'google_

### Defining Utility Function

In [5]:
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)


Utility Function: LinearUtility: Σ(weights * values) where weights = [tf.Tensor(0.6, shape=(), dtype=float32), tf.Tensor(0.4, shape=(), dtype=float32)]


### Defining State Rep State

In [6]:
# Add root factors (with empty relationships)
aicon.add_state_factor(
    name="market_size",
    factor_type="continuous",
    value=10000.0,
    params={
        "loc": 10000.0,
        "scale": 1000.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="competition_level",
    factor_type="categorical",
    value="medium",
    params={
        "categories": ["low", "medium", "high"],
        "probs": [0.2, 0.5, 0.3]
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

# Add dependent factors
aicon.add_state_factor(
    name="conversion_rate",
    factor_type="continuous",
    value=0.02,
    params={
        "loc": 0.02,
        "scale": 0.005,
        "constraints": {"lower": 0, "upper": 1}
    },
    relationships={
        "depends_on": ["market_size", "competition_level"]
    }
)

aicon.add_state_factor(
    name="customer_acquisition_cost",
    factor_type="continuous",
    value=50.0,
    params={
        "loc": 50.0,
        "scale": 10.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": ["competition_level"]
    }
)

# Add a discrete factor for number of competitors
aicon.add_state_factor(
    name="num_competitors",
    factor_type="discrete",
    value=5,
    params={
        "rate": 5.0  # Poisson distribution parameter
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

DiscreteLatentVariable(name='num_competitors', value=5, description='')

In [7]:
print(aicon.brain.state)

State Representation:
market_size: 10000.00 (constraints: {'lower': 0}) (uncertainty: 1000.00)
competition_level: medium (probs: {'low': 0.2, 'medium': 0.5, 'high': 0.3})
conversion_rate: 0.02 (constraints: {'lower': 0, 'upper': 1}) (uncertainty: 0.01)
customer_acquisition_cost: 50.00 (constraints: {'lower': 0}) (uncertainty: 10.00)
num_competitors: 5 (Poisson rate: 5.0)

Factor Relationships:

market_size:
  Root factor (no dependencies)

competition_level:
  Root factor (no dependencies)

conversion_rate:
  Depends on: ['market_size', 'competition_level']

customer_acquisition_cost:
  Depends on: ['competition_level']

num_competitors:
  Root factor (no dependencies)



### Perception

In [7]:
# Try to update beliefs without any sensors
print("Current state before update:")
print(aicon.get_state())

Current state before update:
{'base_conversion_rate': 0.05, 'primary_channel': 'google'}


## Sensors

In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import all necessary modules first
import aicons.bayesbrainGPT.bayes_brain_ref
import aicons.bayesbrainGPT.utility_function.utility_base
import aicons.bayesbrainGPT.utility_function.linear_utility
import aicons.bayesbrainGPT.utility_function
import aicons.definitions.aicon

# Now we can reload them
importlib.reload(aicons.bayesbrainGPT.bayes_brain_ref)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)
importlib.reload(aicons.bayesbrainGPT.utility_function)  # Reload the package first
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT)
importlib.reload(aicons)

# Import the classes we need
from aicons.definitions.aicon import AIcon
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor

# Now we can create the AIcon and proceed with the rest of the code...

In [2]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


In [3]:
# Define an action space first
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [4]:
# Define a utility function
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)


Utility Function: LinearUtility: Σ(weights * values) where weights = [tf.Tensor(0.6, shape=(), dtype=float32), tf.Tensor(0.4, shape=(), dtype=float32)]
